<h1>CSE483 Computer Vision Major Task - Phase 2</h1>
<h2>Team 1</h2>
<ul>
<li>Youssef George 19p9824</li>
<li>Kerollos Wageeh 19p3468</li>
<li>Nada Amr 19p1621</li>
</ul>

## Import Libraries
- Use "pip install libName" in terminal if any of them is not installed on your machine.

In [1]:
import numpy as np
import cv2
from scipy.io import loadmat
import json
import statistics
import torch
from torchvision import ops
from operator import itemgetter
import random
import tensorflow as tf
import matplotlib.pyplot as plt
import scipy
import glob
from skimage.feature import hog
from sklearn.neighbors import KNeighborsClassifier
from collections import Counter
import warnings
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)

## Custom Fns

In [2]:
# Define the HOG parameters
cell_size = (8, 8)
block_size = (16, 16)
orientations = 9

In [3]:
# Fn import pics, labels, true boxes
def getPics(chosen_set):
    images = []
    labels = []
    boxes = []
    
    picsFolder_path = "SVHN/" + chosen_set + "/"
    with open(picsFolder_path + 'digitStruct.json') as f:
        data = json.load(f)
    with open(picsFolder_path + 'phase1Output.json') as f:
        selected_boxes = json.load(f)

# import colored pictures
    for i in range(len(data)):
        image = cv2.imread(picsFolder_path + data[i]['filename'])
        image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
        images.append(image)
        temp=[]
        for j in range(len(data[i]['boxes'])):
            temp.append(data[i]['boxes'][j]['label'])
        temp = np.array(temp)
        labels.append(temp)
        boxes.append(selected_boxes[i]['selected_boxes'+str(i)])

    print("There are ", len(data), " images in " + chosen_set + " set.")
    return images, labels, boxes

In [4]:
def getPicsCropped(chosen_set):
    images = []
    gs_images = []
    labels = []
    boxes = []

    picsFolder_path = "SVHN/" + chosen_set + "_32x32.mat"
    data = scipy.io.loadmat(picsFolder_path)
    images = data['X']
    labels = data['y']
    images = np.array(images)
    images = np.moveaxis(images, -1, 0)
    gs_images = np.array(gs_images)
    gs_images = np.moveaxis(gs_images, -1, 0)
    labels = np.array(labels)
    print(images.shape[0],"cropped images in 32x32 " + chosen_set + " set.")
    return images, labels

In [5]:
train_images_all, train_labels_all, train_boxes_all = getPics("train")

There are  33402  images in train set.


In [6]:
train_boxes_all[1]

[[100, 28, 121, 57]]

In [14]:
def cropImages(image, boxes):
    img = image.copy()
    img = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
    cropped = []
    for box in boxes:
        cropped.append(img[box[1]:box[3], box[0]:box[2]])
    return cropped

In [15]:
def getTemplatesPics():
    templates = []
    labels = []
    picsFolder_path = "Matching/"

    srtd_dir = sorted(glob.glob(picsFolder_path + "/*.jpg"))
    # import colored pictures
    for image_path in srtd_dir:
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
        template = {'label':int(image_path[9]), 'image':image}
        templates.append(template)
    return templates

In [16]:
def extract_hog_features(image):
    resized_image = cv2.resize(image, (64, 128))
    hog = cv2.HOGDescriptor()
    hog_features = hog.compute(resized_image)
    hog_features = np.array(hog_features)
    return hog_features

In [17]:
def getTemplatesHOGs():
    templates = getTemplatesPics()
    full_templates = []
    for template in templates:
        template['descriptor'] = extract_hog_features(template['image'])
        full_templates.append(template)
    return full_templates

In [18]:
def calcPicRecall(trueLabels, predictedLabels):
    truePredictions = list((Counter(trueLabels) & Counter(predictedLabels)).elements())
    recall = len(truePredictions)/len(predictedLabels)
    return recall

In [19]:
def predictImage(image, boxes, knn):
    croppedImages = cropImages(image,boxes)
    predictions = []
    for img in croppedImages:
        predictions.append(knn.predict(extract_hog_features(img)))
    return predictions

In [20]:
templates = getTemplatesHOGs()
tmps_data = [temp['descriptor'] for temp in templates]
tmps_labels = [temp['label'] for temp in templates]
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(tmps_data, tmps_labels)

acc = []
for image, boxes, label in zip(train_images_all, train_boxes_all, train_labels_all):
    if(len(boxes)==0):
        continue
    predictImage(image,boxes,knn)
#     x = calcPicRecall(label.tolist(), predictImage(image, boxes, knn))
#     acc.append(x)
# print(np.average(acc))

ValueError: Expected 2D array, got 1D array instead:
array=[0.08298624 0.05061684 0.03445257 ... 0.00125127 0.00041815 0.00706484].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [ ]:


acc = []

calcPicRecall
    

true label: [1. 7.]
[7]
[1]


In [ ]:
acc = sorted(acc, reverse=True)
print(acc[460:1000])

[1.0, 0.75, 0.75, 0.75, 0.75, 0.6666666666666666, 0.6666666666666666, 0.6666666666666666, 0.6666666666666666, 0.6666666666666666, 0.6666666666666666, 0.6666666666666666, 0.6666666666666666, 0.6666666666666666, 0.6666666666666666, 0.6666666666666666, 0.6666666666666666, 0.6666666666666666, 0.6666666666666666, 0.6666666666666666, 0.6666666666666666, 0.6666666666666666, 0.6666666666666666, 0.6666666666666666, 0.6666666666666666, 0.6666666666666666, 0.6666666666666666, 0.6666666666666666, 0.6666666666666666, 0.6666666666666666, 0.6666666666666666, 0.6666666666666666, 0.6666666666666666, 0.6666666666666666, 0.6666666666666666, 0.6666666666666666, 0.6666666666666666, 0.6666666666666666, 0.6666666666666666, 0.6666666666666666, 0.6666666666666666, 0.6666666666666666, 0.6666666666666666, 0.6666666666666666, 0.6666666666666666, 0.6666666666666666, 0.6666666666666666, 0.6666666666666666, 0.6666666666666666, 0.6666666666666666, 0.6666666666666666, 0.6666666666666666, 0.6666666666666666, 0.66666666

------------------------------